In [1]:
from concurrent.futures import ThreadPoolExecutor
import time
from langchain_anthropic import ChatAnthropic
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import json
import re
from pygments import highlight
from pygments.lexers import JsonLexer
from pygments.formatters import HtmlFormatter
from IPython.display import HTML
import pandas as pd
# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import pandas as pd
from openai import OpenAI
from typing import List, Tuple, Union, Dict, Any
import ast

import re
import json
import glob


pd.set_option('display.max_colwidth', 500)

llm_api_key = "sk-gapk-GHjvOA9hXL8MQ7yNNlR7kLmfA-f8fSl6"  #우리꺼
# llm_api_key = "sk-gapk-1tQCB4O2KnH5GG68DeUKfjAKQ-vJ9kc9" #도현님
# llm_api_key = "sk-gapk-Y70vdkPbXPRMWHK0dtaYU30hw-bi7B5C" # 빌린거
llm_api_url = "https://api.platform.a15t.com/v1"


client = OpenAI(
    api_key = llm_api_key,
    base_url = llm_api_url
)

# from langchain.chat_models import ChatOpenAI
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from langchain.schema import AIMessage, HumanMessage, SystemMessage
import pandas as pd

def ChatAnthropicSKT(model="skt/claude-3-7-sonnet-20250219", max_tokens=4000):
    llm_api_key = "sk-gapk-GHjvOA9hXL8MQ7yNNlR7kLmfA-f8fSl6" #우리꺼 # "sk-gapk-1tQCB4O2KnH5GG68DeUKfjAKQ-vJ9kc9"
    # llm_api_key = "sk-gapk-1tQCB4O2KnH5GG68DeUKfjAKQ-vJ9kc9" #도현님  # "sk-gapk-1tQCB4O2KnH5GG68DeUKfjAKQ-vJ9kc9"
    # llm_api_key = "sk-gapk-Y70vdkPbXPRMWHK0dtaYU30hw-bi7B5C" # 빌린거
    
    llm_api_url = "https://api.platform.a15t.com/v1"
    
    # llm_api_url = "https://43.203.77.11:443/v1"

    # model = "anthropic/claude-3-5-sonnet-20240620"

    model = ChatOpenAI(
        temperature=0,  
        openai_api_key=llm_api_key, 
        openai_api_base=llm_api_url, 
        model=model,
        max_tokens=max_tokens
        )
    return model

llm_cld37 = ChatAnthropicSKT()
llm_gem3 = ChatAnthropicSKT(model='skt/gemma3-12b-it-dev')
# llm_ax = ChatAnthropicSKT(model='skt/a.x-3-lg')


# llm_cld37 = ChatAnthropic(
#     api_key=os.getenv("ANTHROPIC_API_KEY"),
#     model="claude-3-7-sonnet-20250219",
#     max_tokens=3000
# )

llm_chat = ChatOpenAI(
        temperature=0,  
        model="gpt-4.1",
        openai_api_key=os.getenv("OPENAI_API_KEY"),
        max_tokens=2000,
)

llm_cld40 = ChatAnthropic(
    api_key=os.getenv("ANTHROPIC_API_KEY"),
    model="claude-sonnet-4-20250514",
    max_tokens=3000
)


In [2]:
from typing import List, Tuple, Union, Dict, Any
import ast

import re
import json
import glob

def dataframe_to_markdown_prompt(df, max_rows=None):
    # Limit rows if needed
    if max_rows is not None and len(df) > max_rows:
        display_df = df.head(max_rows)
        truncation_note = f"\n[Note: Only showing first {max_rows} of {len(df)} rows]"
    else:
        display_df = df
        truncation_note = ""
    
    # Convert to markdown
    df_markdown = display_df.to_markdown()
    
    prompt = f"""

    {df_markdown}
    {truncation_note}

    """
    
    return prompt

def replace_strings(text, replacements):
    for old, new in replacements.items():
        text = text.replace(old, new)
        
    return text

def clean_segment(segment):
    """
    Given a segment that is expected to be quoted (i.e. begins and ends with
    the same single or double quote), remove any occurrences of that quote
    from the inner content.
    For example, if segment is:
         "에이닷 T 멤버십 쿠폰함에 "에이닷은통화요약된닷" 입력"
    then the outer quotes are preserved but the inner double quotes are removed.
    """
    segment = segment.strip()
    if len(segment) >= 2 and segment[0] in ['"', "'"] and segment[-1] == segment[0]:
        q = segment[0]
        # Remove inner occurrences of the quote character.
        inner = segment[1:-1].replace(q, '')
        return q + inner + q
    return segment

def split_key_value(text):
    """
    Splits text into key and value based on the first colon that appears
    outside any quoted region.
    If no colon is found outside quotes, the value will be returned empty.
    """
    in_quote = False
    quote_char = ''
    for i, char in enumerate(text):
        if char in ['"', "'"]:
            # Toggle quote state (assumes well-formed starting/ending quotes for each token)
            if in_quote:
                if char == quote_char:
                    in_quote = False
                    quote_char = ''
            else:
                in_quote = True
                quote_char = char
        elif char == ':' and not in_quote:
            return text[:i], text[i+1:]
    return text, ''

def split_outside_quotes(text, delimiter=','):
    """
    Splits the input text on the given delimiter (default comma) but only
    if the delimiter occurs outside of quoted segments.
    Returns a list of parts.
    """
    parts = []
    current = []
    in_quote = False
    quote_char = ''
    for char in text:
        if char in ['"', "'"]:
            # When encountering a quote, toggle our state
            if in_quote:
                if char == quote_char:
                    in_quote = False
                    quote_char = ''
            else:
                in_quote = True
                quote_char = char
            current.append(char)
        elif char == delimiter and not in_quote:
            parts.append(''.join(current).strip())
            current = []
        else:
            current.append(char)
    if current:
        parts.append(''.join(current).strip())
    return parts

def clean_ill_structured_json(text):
    """
    Given a string that is intended to represent a JSON-like structure
    but may be ill-formed (for example, it might contain nested quotes that
    break standard JSON rules), attempt to “clean” it by processing each
    key–value pair.
    
    The function uses the following heuristics:
      1. Split the input text into comma-separated parts (only splitting
         when the comma is not inside a quoted string).
      2. For each part, split on the first colon (that is outside quotes) to separate key and value.
      3. For any segment that begins and ends with a quote, remove any inner occurrences
         of that same quote.
      4. Rejoin the cleaned key and value.
    
    Note: This approach does not build a fully robust JSON parser. For very complex
          or deeply nested ill-structured inputs further refinement would be needed.
    """
    # First, split the text by commas outside of quotes.
    parts = split_outside_quotes(text, delimiter=',')
    
    cleaned_parts = []
    for part in parts:
        # Try to split into key and value on the first colon not inside quotes.
        key, value = split_key_value(part)
        key_clean = clean_segment(key)
        value_clean = clean_segment(value) if value.strip() != "" else ""
        if value_clean:
            cleaned_parts.append(f"{key_clean}: {value_clean}")
        else:
            cleaned_parts.append(key_clean)
    
    # Rejoin the cleaned parts with commas (or you can use another format if desired)
    return ', '.join(cleaned_parts)

def repair_json(broken_json):
    
    # json_str = broken_json.replace("'",'"')
    
    # Fix unquoted values (like NI00001863)
    json_str = re.sub(r':\s*([a-zA-Z0-9_]+)(\s*[,}])', r': "\1"\2', broken_json)
    
    # Fix unquoted keys
    json_str = re.sub(r'([{,])\s*([a-zA-Z0-9_]+):', r'\1 "\2":', json_str)
    
    # Fix trailing commas
    json_str = re.sub(r',\s*}', '}', json_str)
    
    return json_str

def extract_json_objects(text):
    # More sophisticated pattern that tries to match proper JSON syntax
    pattern = r'(\{(?:[^{}]|(?:\{(?:[^{}]|(?:\{[^{}]*\}))*\}))*\})'
    
    result = []
    for match in re.finditer(pattern, text):
        potential_json = match.group(0)
        try:
            # Try to parse and validate
            # json_obj = json.loads(repair_json(potential_json))
            json_obj = ast.literal_eval(clean_ill_structured_json(repair_json(potential_json)))
            result.append(json_obj)
        except json.JSONDecodeError:
            # Not valid JSON, skip
            pass
    
    return result

def extract_between(text, start_marker, end_marker):
    start_index = text.find(start_marker)
    if start_index == -1:
        return None
    
    start_index += len(start_marker)
    end_index = text.find(end_marker, start_index)
    if end_index == -1:
        return None
    
    return text[start_index:end_index]

def extract_content(text: str, tag_name: str) -> List[str]:
    pattern = f'<{tag_name}>(.*?)</{tag_name}>'
    matches = re.findall(pattern, text, re.DOTALL)
    return matches


def clean_bad_text(text):
    import re
    
    if not isinstance(text, str):
        return ""
    
    # Remove URLs and emails
    text = re.sub(r'https?://\S+|www\.\S+', ' ', text)
    text = re.sub(r'\S+@\S+', ' ', text)
    
    # Keep Korean, alphanumeric, spaces, and specific punctuation
    text = re.sub(r'[^\uAC00-\uD7A3\u1100-\u11FF\w\s\.\?!,]', ' ', text)
    
    # Normalize whitespace
    text = re.sub(r'\s+', ' ', text).strip()
    
    return text

def clean_text(text):
    """
    Cleans text by removing special characters that don't affect fine-tuning.
    Preserves important structural elements like quotes, brackets, and JSON syntax.
    Specifically handles Korean text (Hangul) properly.
    
    Args:
        text (str): The input text to clean
        
    Returns:
        str: Cleaned text ready for fine-tuning
    """
    import re
    
    # Preserve the basic structure by temporarily replacing important characters
    # with placeholder tokens that won't be affected by cleanup
    
    # Step 1: Temporarily replace JSON structural elements
    placeholders = {
        '"': "DQUOTE_TOKEN",
        "'": "SQUOTE_TOKEN",
        "{": "OCURLY_TOKEN",
        "}": "CCURLY_TOKEN",
        "[": "OSQUARE_TOKEN",
        "]": "CSQUARE_TOKEN",
        ":": "COLON_TOKEN",
        ",": "COMMA_TOKEN"
    }
    
    for char, placeholder in placeholders.items():
        text = text.replace(char, placeholder)
    
    # Step 2: Remove problematic characters
    
    # Remove control characters (except newlines, carriage returns, and tabs which can be meaningful)
    text = re.sub(r'[\x00-\x08\x0B\x0C\x0E-\x1F\x7F]', '', text)
    
    # Normalize all types of newlines to \n
    text = re.sub(r'\r\n|\r', '\n', text)
    
    # Remove zero-width characters and other invisible unicode
    text = re.sub(r'[\u200B-\u200D\uFEFF\u00A0]', '', text)
    
    # MODIFIED: Keep Korean characters (Hangul) along with other useful character sets
    # This regex keeps:
    # - ASCII (Basic Latin): \x00-\x7F
    # - Latin-1 Supplement: \u0080-\u00FF
    # - Latin Extended A & B: \u0100-\u017F\u0180-\u024F
    # - Greek and Coptic: \u0370-\u03FF
    # - Cyrillic: \u0400-\u04FF
    # - Korean Hangul Syllables: \uAC00-\uD7A3
    # - Hangul Jamo (Korean alphabet): \u1100-\u11FF
    # - Hangul Jamo Extended-A: \u3130-\u318F
    # - Hangul Jamo Extended-B: \uA960-\uA97F
    # - Hangul Compatibility Jamo: \u3130-\u318F
    # - CJK symbols and punctuation: \u3000-\u303F
    # - Full-width forms (often used with CJK): \uFF00-\uFFEF
    # - CJK Unified Ideographs (Basic common Chinese/Japanese characters): \u4E00-\u9FFF
    
    # Instead of removing characters, we'll define which ones to keep
    allowed_chars_pattern = r'[^\x00-\x7F\u0080-\u00FF\u0100-\u024F\u0370-\u03FF\u0400-\u04FF' + \
                           r'\u1100-\u11FF\u3130-\u318F\uA960-\uA97F\u3000-\u303F' + \
                           r'\uAC00-\uD7A3\uFF00-\uFFEF\u4E00-\u9FFF\n\r\t ]'
    text = re.sub(allowed_chars_pattern, '', text)
    
    # Step 3: Normalize whitespace (but preserve deliberate line breaks)
    text = re.sub(r'[ \t]+', ' ', text)  # Convert multiple spaces/tabs to single space
    
    # First ensure all newlines are standardized
    text = re.sub(r'\r\n|\r', '\n', text)  # Convert all newline variants to \n
    
    # Then normalize multiple blank lines to at most two
    text = re.sub(r'\n\s*\n+', '\n\n', text)  # Convert multiple newlines to at most two
    
    # Step 4: Restore original JSON structural elements
    for char, placeholder in placeholders.items():
        text = text.replace(placeholder, char)
    
    # Step 5: Fix common JSON syntax issues that might remain
    # Fix spaces between quotes and colons in JSON
    text = re.sub(r'"\s+:', r'":', text)
    
    # Fix trailing commas in arrays
    text = re.sub(r',\s*]', r']', text)
    
    # Fix trailing commas in objects
    text = re.sub(r',\s*}', r'}', text)
    
    return text

def remove_control_characters(text):
    if isinstance(text, str):
        # Remove control characters except commonly used whitespace
        return re.sub(r'[\x00-\x08\x0B-\x0C\x0E-\x1F\x7F]', '', text)
    return text

import openai
from langchain.chains import RetrievalQA
from langchain.llms.openai import OpenAIChat  # For compatibility with newer setup

# Create a custom LLM class that uses the OpenAI client directly
class CustomOpenAI:
    def __init__(self, model="skt/a.x-3-lg"):
        self.model = model
        
    def __call__(self, prompt):
        response = client.chat.completions.create(
            model=self.model,
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.1
        )
        return response.choices[0].message.content

# Create a simple retrieval function
def get_relevant_context(query, vectorstore, topk=5):
    docs = vectorstore.similarity_search(query, k=topk)
    context = "\n\n".join([doc.page_content for doc in docs])
    titles = ", ".join(set([doc.metadata['title'] for doc in docs if 'title' in doc.metadata.keys()]))
    return {'title':titles, 'context':context}

# Create a function to combine everything
def answer_question(query, vectorstore):
    # Get relevant context
    context = get_relevant_context(query, vectorstore)
    
    # Create combined prompt
    prompt = f"Answer the following question based on the provided context:\n\nContext: {context}\n\nQuestion: {query}\n\nAnswer:"
    
    # Use OpenAI directly
    custom_llm = CustomOpenAI()  # Or your preferred model
    response = custom_llm(prompt)
    
    return response

def is_list_of_dicts(var):
    # Check if the variable is a list
    if not isinstance(var, list):
        return False
    
    # Check if the list is not empty and all elements are dictionaries
    if not var:  # Empty list
        return False
        
    # Check that all elements are dictionaries
    return all(isinstance(item, dict) for item in var)

def remove_duplicate_dicts(dict_list):
    result = []
    seen = set()
    for d in dict_list:
        # Convert dictionary to a hashable tuple of items
        t = tuple(sorted(d.items()))
        if t not in seen:
            seen.add(t)
            result.append(d)
    return result

def convert_to_custom_format(json_items):
    custom_format = []
    
    for item in json_items:
        item_name = item.get("item_name_in_message", "")
        item_id = item.get("item_id", "")
        category = item.get("category", "")
        
        # Create custom format for each item
        custom_line = f"[Item Name] {item_name} [Item ID] {item_id} [Item Category] {category}"
        custom_format.append(custom_line)
    
    return "\n".join(custom_format)

def remove_urls(text):
    # Regular expression pattern to match URLs
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    
    # Replace URLs with an empty string
    return url_pattern.sub('', text)

def remove_custom_pattern(text, keyword="바로가기"):
    # Create a pattern that matches any text followed by the specified keyword
    # We escape the keyword to handle any special regex characters it might contain
    escaped_keyword = re.escape(keyword)
    pattern = re.compile(r'.*? ' + escaped_keyword)
    
    # Replace the matched pattern with an empty string
    return pattern.sub('', text)

def select_most_comprehensive(strings):
    """
    Select the most comprehensive string from a list of overlapping strings.
    Returns the longest string that contains other strings as substrings.
    
    Args:
        strings: List of strings to filter
        
    Returns:
        List of most comprehensive strings (usually one, but could be multiple if no containment)
    """
    if not strings:
        return []
    
    # Remove duplicates and sort by length (longest first)
    unique_strings = list(set(strings))
    unique_strings.sort(key=len, reverse=True)
    
    result = []
    
    for current in unique_strings:
        # Check if current string contains any of the strings already in result
        is_contained = any(current in existing for existing in result)
        
        # Check if current string contains other strings not yet in result
        contains_others = any(other in current for other in unique_strings if other != current and other not in result)
        
        # If current is not contained by existing results and either:
        # 1. It contains other strings, or 
        # 2. No strings contain each other (keep all unique)
        if not is_contained:
            # Remove any strings from result that are contained in current
            result = [r for r in result if r not in current]
            result.append(current)
    
    return result

In [3]:
import spacy
import re
from typing import List, Dict, Set, Tuple, Any
from collections import defaultdict

class KoreanNERSpacy:
    def __init__(self, entity_vocab: List[Tuple[str, Dict[str, Any]]]):
        """
        Initialize with new entity vocabulary schema
        entity_vocab: List of (alias, metadata) tuples where metadata contains:
        - item_nm: canonical name
        - item_id: unique identifier  
        - description: entity description
        - domain: entity domain/category
        - item_nm_alias: display alias
        """
        self.entity_vocab = entity_vocab
        self.nlp = spacy.load("ko_core_news_sm")
        
        # Build efficient lookup structures
        self.alias_to_metadata = {}
        self.canonical_to_metadata = {}
        self.aliases_by_canonical = defaultdict(list)
        self.entities_by_domain = defaultdict(list)
        self.entities_by_length = defaultdict(list)
        self.all_aliases = set()
        
        self._build_lookup_structures()
        
        # Pre-compute normalized variants for fuzzy matching
        self.normalized_mappings = self._create_normalized_mappings()

    def _build_lookup_structures(self):
        """Build efficient lookup structures from the entity vocab"""
        for alias, metadata in self.entity_vocab:
            # Skip invalid entries (like '#')
            if not alias or alias == '#':
                continue
                
            canonical_name = metadata['item_nm']
            domain = metadata['domain']
            
            # Store mappings
            self.alias_to_metadata[alias] = metadata
            self.canonical_to_metadata[canonical_name] = metadata
            self.aliases_by_canonical[canonical_name].append(alias)
            self.entities_by_domain[domain].append(alias)
            self.entities_by_length[len(alias)].append(alias)
            self.all_aliases.add(alias)
        
        # Sort lengths for efficient processing (longest first)
        self.sorted_lengths = sorted(self.entities_by_length.keys(), reverse=True)

    def _create_normalized_mappings(self) -> Dict[str, str]:
        """Create normalized forms for fuzzy matching"""
        mappings = {}
        for alias in self.all_aliases:
            # Multiple normalization strategies for Korean
            normalized_forms = [
                re.sub(r'\s+', '', alias.lower()),  # Remove spaces
                re.sub(r'[^\w가-힣]', '', alias.lower()),  # Remove special chars
                alias.lower().replace('ipad', '아이패드').replace('iphone', '아이폰'),  # EN->KR
                alias.lower().replace('아이패드', 'ipad').replace('아이폰', 'iphone'),  # KR->EN
                re.sub(r'프로|PRO', 'pro', alias.lower(), flags=re.IGNORECASE),  # Normalize "pro"
            ]
            
            for normalized in normalized_forms:
                if normalized and normalized != alias.lower():
                    mappings[normalized] = alias
                    
        return mappings

    def extract_entities(self, text: str, method: str = "optimized") -> List[Dict]:
        """Main entity extraction method"""
        if method == "optimized":
            return self._extract_optimized(text)
        elif method == "fuzzy":
            return self._extract_fuzzy(text)
        elif method == "hybrid":
            return self._extract_hybrid(text)
        else:
            return self._extract_exact(text)

    def _extract_exact(self, text: str) -> List[Dict]:
        """Enhanced exact matching with comprehensive case handling"""
        results = []
        processed_positions = set()
        
        # Step 1: Exact case-sensitive matching (highest priority)
        for length in self.sorted_lengths:
            if length > len(text):
                continue
                
            aliases = self.entities_by_length[length]
            for alias in aliases:
                start = 0
                while True:
                    pos = text.find(alias, start)  # Case-sensitive
                    if pos == -1:
                        break
                    
                    if any(p in processed_positions for p in range(pos, pos + length)):
                        start = pos + 1
                        continue
                    
                    metadata = self.alias_to_metadata[alias]
                    results.append({
                        "text": text[pos:pos + length],
                        "alias": alias,
                        "canonical_name": metadata['item_nm'],
                        "item_id": metadata['item_id'],
                        "domain": metadata['domain'],
                        "description": metadata['description'],
                        "start": pos,
                        "end": pos + length,
                        "confidence": 1.0,
                        "method": "exact_case_match",
                        "case_match": "exact"
                    })
                    
                    for p in range(pos, pos + length):
                        processed_positions.add(p)
                    start = pos + 1
        
        # Step 2: Case-insensitive matching for remaining text
        text_lower = text.lower()
        for length in self.sorted_lengths:
            if length > len(text):
                continue
                
            aliases = self.entities_by_length[length]
            for alias in aliases:
                alias_lower = alias.lower()
                start = 0
                
                while True:
                    pos = text_lower.find(alias_lower, start)
                    if pos == -1:
                        break
                    
                    if any(p in processed_positions for p in range(pos, pos + length)):
                        start = pos + 1
                        continue
                    
                    # Check if this is actually a case variation (not exact match)
                    original_text = text[pos:pos + length]
                    if original_text != alias:  # Only add if case differs
                        metadata = self.alias_to_metadata[alias]
                        case_type = self._analyze_case_type(original_text, alias)
                        
                        results.append({
                            "text": original_text,
                            "alias": alias,
                            "canonical_name": metadata['item_nm'],
                            "item_id": metadata['item_id'],
                            "domain": metadata['domain'],
                            "description": metadata['description'],
                            "start": pos,
                            "end": pos + length,
                            "confidence": 0.9,
                            "method": "case_insensitive_match",
                            "case_match": case_type
                        })
                        
                        for p in range(pos, pos + length):
                            processed_positions.add(p)
                    
                    start = pos + 1
        
        return sorted(results, key=lambda x: x['start'])
    
    def _analyze_case_type(self, found_text: str, original_alias: str) -> str:
        """Analyze the type of case difference"""
        if found_text.isupper():
            return "all_upper"
        elif found_text.islower():
            return "all_lower"
        elif found_text.istitle():
            return "title_case"
        else:
            return "mixed_case"

    def _extract_fuzzy(self, text: str) -> List[Dict]:
        """Fuzzy matching for Korean variations with deduplication"""
        results = []
        
        # First do exact matching
        exact_results = self._extract_exact(text)
        results.extend(exact_results)
        
        # Get processed positions from exact matches
        processed_positions = set()
        found_entities = set()  # Track unique entities
        
        for result in exact_results:
            for pos in range(result['start'], result['end']):
                processed_positions.add(pos)
            # Add to found entities set
            entity_key = (result['start'], result['end'], result['canonical_name'])
            found_entities.add(entity_key)
        
        # Try normalized matching for unprocessed parts
        doc = self.nlp(text)
        
        for token in doc:
            # Skip if this token overlaps with exact matches
            if any(pos in processed_positions for pos in range(token.idx, token.idx + len(token.text))):
                continue

            # Try various normalizations
            token_text = token.text
            normalized_forms = [
                re.sub(r'\s+', '', token_text.lower()),
                re.sub(r'[^\w가-힣]', '', token_text.lower()),
                token_text.lower(),
            ]
            
            for normalized in normalized_forms:
                if normalized in self.normalized_mappings:
                    original_alias = self.normalized_mappings[normalized]
                    metadata = self.alias_to_metadata[original_alias]
                    
                    # Create unique entity key
                    entity_key = (token.idx, token.idx + len(token.text), metadata['item_nm'])
                    
                    # Skip if we already found this entity at this position
                    if entity_key in found_entities:
                        continue
                    
                    results.append({
                        "text": token_text,
                        "alias": original_alias,
                        "canonical_name": metadata['item_nm'],
                        "item_id": metadata['item_id'],
                        "domain": metadata['domain'],
                        "description": metadata['description'],
                        "start": token.idx,
                        "end": token.idx + len(token.text),
                        "confidence": 0.85,
                        "method": "fuzzy_match"
                    })
                    
                    found_entities.add(entity_key)
                    break  # Stop after first match for this token
        
        return self._deduplicate_method_results(results)

    def _extract_optimized(self, text: str) -> List[Dict]:
        """Optimized extraction combining exact + smart fuzzy matching with deduplication"""
        results = []
        found_entities = set()  # Global entity tracking
        
        # Step 1: Exact matching (highest priority)
        exact_results = self._extract_exact(text)
        results.extend(exact_results)
        
        # Track found entities
        for result in exact_results:
            entity_key = (result['start'], result['end'], result['canonical_name'])
            found_entities.add(entity_key)
        
        # Step 2: Handle Korean-specific patterns in remaining text
        processed_positions = set()
        for result in exact_results:
            for pos in range(result['start'], result['end']):
                processed_positions.add(pos)
        
        # Step 3: Multi-token matching for compound device names
        doc = self.nlp(text)
        tokens = [token for token in doc if not token.is_space]
        
        # Try 2-4 token combinations
        for i in range(len(tokens)):
            for j in range(i + 2, min(i + 5, len(tokens) + 1)):
                # Skip if any token in range is already processed
                token_range = tokens[i:j]
                if any(any(pos in processed_positions 
                          for pos in range(token.idx, token.idx + len(token.text))) 
                       for token in token_range):
                    continue
                
                start_pos = tokens[i].idx
                end_pos = tokens[j-1].idx + len(tokens[j-1].text)
                
                # Create entity key for this position range
                entity_key_base = (start_pos, end_pos)
                
                # Create candidate strings
                candidates = [
                    "".join([t.text for t in token_range]),  # No spaces
                    " ".join([t.text for t in token_range]), # With spaces
                ]
                
                found_match = False
                for candidate in candidates:
                    if candidate.lower() in [alias.lower() for alias in self.all_aliases]:
                        # Find the original alias
                        original_alias = next(alias for alias in self.all_aliases 
                                            if alias.lower() == candidate.lower())
                        metadata = self.alias_to_metadata[original_alias]
                        
                        # Create unique entity key
                        entity_key = (start_pos, end_pos, metadata['item_nm'])
                        
                        # Skip if we already found this entity at this position
                        if entity_key in found_entities:
                            continue
                        
                        results.append({
                            "text": text[start_pos:end_pos],
                            "alias": original_alias,
                            "canonical_name": metadata['item_nm'],
                            "item_id": metadata['item_id'],
                            "domain": metadata['domain'],
                            "description": metadata['description'],
                            "start": start_pos,
                            "end": end_pos,
                            "confidence": 0.9,
                            "method": "multi_token_match"
                        })
                        
                        # Mark as processed and found
                        for token in token_range:
                            for pos in range(token.idx, token.idx + len(token.text)):
                                processed_positions.add(pos)
                        found_entities.add(entity_key)
                        found_match = True
                        break
                
                if found_match:
                    break  # Don't try longer combinations if we found a match
        
        return self._deduplicate_method_results(results)

    def _extract_hybrid(self, text: str) -> List[Dict]:
        """Hybrid approach combining all methods with comprehensive deduplication"""
        all_results = []
        global_found_entities = set()  # Track entities across all methods
        
        # Method 1: Optimized extraction
        optimized_results = self._extract_optimized(text)
        all_results.extend(optimized_results)
        
        # Track entities from optimized method
        for result in optimized_results:
            entity_key = (result['start'], result['end'], result['canonical_name'])
            global_found_entities.add(entity_key)
        
        # Method 2: Additional fuzzy results for uncovered areas
        fuzzy_results = self._extract_fuzzy(text)
        
        # Only add fuzzy results that don't duplicate existing entities
        for result in fuzzy_results:
            entity_key = (result['start'], result['end'], result['canonical_name'])
            if entity_key not in global_found_entities:
                all_results.append(result)
                global_found_entities.add(entity_key)
        
        return self._deduplicate_method_results(all_results)

    def _deduplicate_method_results(self, results: List[Dict]) -> List[Dict]:
        """Enhanced deduplication for method-level results"""
        if not results:
            return results
        
        # Step 1: Remove exact duplicates by creating unique keys
        seen_entities = {}  # key -> best_result
        
        for result in results:
            # Create comprehensive unique key
            unique_key = (
                result['start'],
                result['end'], 
                result['canonical_name'],
                result['item_id']
            )
            
            # Keep the result with highest confidence, or first one if tied
            if (unique_key not in seen_entities or 
                result['confidence'] > seen_entities[unique_key]['confidence']):
                seen_entities[unique_key] = result
        
        # Step 2: Handle overlapping entities (keep highest confidence)
        deduplicated_results = list(seen_entities.values())
        return self._remove_overlaps(deduplicated_results)

    def _remove_overlaps(self, results: List[Dict]) -> List[Dict]:
        """Enhanced overlap removal with better prioritization"""
        if not results:
            return results
        
        # Sort by multiple criteria for better conflict resolution
        def sort_key(result):
            # Priority: confidence, exactness of method, length, position
            method_priority = {
                "exact_case_match": 4,
                "case_insensitive_match": 3,
                "multi_token_match": 2,
                "fuzzy_match": 1
            }
            
            return (
                -result['confidence'],                                    # Higher confidence first
                -method_priority.get(result['method'], 0),               # Better methods first
                -(result['end'] - result['start']),                     # Longer entities first
                result['start']                                          # Earlier position first
            )
        
        sorted_results = sorted(results, key=sort_key)
        final_results = []
        
        for result in sorted_results:
            # Check for overlap with existing results
            overlap = False
            for existing in final_results:
                if (result['start'] < existing['end'] and result['end'] > existing['start']):
                    overlap = True
                    break
            
            if not overlap:
                final_results.append(result)
        
        return sorted(final_results, key=lambda x: x['start'])

    def get_entities_by_domain(self, domain: str) -> List[str]:
        """Get all entities in a specific domain"""
        return self.entities_by_domain.get(domain, [])
    
    def get_canonical_variants(self, canonical_name: str) -> List[str]:
        """Get all aliases for a canonical name"""
        return self.aliases_by_canonical.get(canonical_name, [])



In [4]:
from rapidfuzz import fuzz, process
import re

class KoreanEntityMatcher:
    def __init__(self, min_similarity=70, ngram_size=2, min_entity_length=2, token_similarity=True):
        self.min_similarity = min_similarity
        self.ngram_size = ngram_size
        self.min_entity_length = min_entity_length
        self.token_similarity = token_similarity  # 토큰 단위 유사도 비교 옵션 추가
        self.entities = []
        self.entity_data = {}
        
    def build_from_list(self, entities):
        """Build entity index from a list of entities"""
        self.entities = []
        self.entity_data = {}
        
        for i, entity in enumerate(entities):
            if isinstance(entity, tuple) and len(entity) == 2:
                entity_name, data = entity
                self.entities.append(entity_name)
                self.entity_data[entity_name] = data
            else:
                self.entities.append(entity)
                self.entity_data[entity] = {'id': i, 'entity': entity}
                
        # 각 엔티티의 정규화된 형태를 저장 (검색 최적화)
        self.normalized_entities = {}
        for entity in self.entities:
            normalized = self._normalize_text(entity)
            self.normalized_entities[normalized] = entity
                
        # Create n-gram index for faster candidate selection
        self._build_ngram_index(n=self.ngram_size)
    
    def _normalize_text(self, text):
        """텍스트 정규화 - 소문자 변환, 공백 제거 등"""
        # 소문자로 변환
        text = text.lower()
        # 연속된 공백을 하나로 통일
        text = re.sub(r'\s+', ' ', text)
        return text.strip()
    
    def _tokenize(self, text):
        """텍스트를 토큰으로 분리 (한글, 영문, 숫자 분리)"""
        # 한글, 영문, 숫자 토큰 추출
        tokens = re.findall(r'[가-힣]+|[a-z0-9]+', self._normalize_text(text))
        return tokens
    
    def _build_ngram_index(self, n=2):
        """Build n-gram index optimized for Korean characters"""
        self.ngram_index = {}
        
        for entity in self.entities:
            # Skip entities shorter than min_entity_length
            if len(entity) < self.min_entity_length:
                continue
                
            # 정규화된 엔티티 사용
            normalized_entity = self._normalize_text(entity)
            
            # Create n-grams for the entity
            entity_chars = list(normalized_entity)  # Split into characters for proper Korean handling
            ngrams = []
            
            # Create character-level n-grams (better for Korean)
            for i in range(len(entity_chars) - n + 1):
                ngram = ''.join(entity_chars[i:i+n])
                ngrams.append(ngram)
            
            # Add entity to the index for each n-gram
            for ngram in ngrams:
                if ngram not in self.ngram_index:
                    self.ngram_index[ngram] = set()
                self.ngram_index[ngram].add(entity)
                
            # 토큰 기반 n-gram도 추가 (실험적)
            tokens = self._tokenize(normalized_entity)
            for token in tokens:
                if len(token) >= n:
                    token_key = f"TOKEN:{token}"
                    if token_key not in self.ngram_index:
                        self.ngram_index[token_key] = set()
                    self.ngram_index[token_key].add(entity)
    
    def _get_candidates(self, text, n=None):
        """Get candidate entities based on n-gram overlap (optimized for Korean)"""
        if n is None:
            n = self.ngram_size
            
        # 텍스트 정규화
        normalized_text = self._normalize_text(text)
        
        # 정규화된 텍스트가 정확히 일치하는지 확인 (빠른 경로)
        if normalized_text in self.normalized_entities:
            entity = self.normalized_entities[normalized_text]
            return [(entity, float('inf'))]  # 정확한 일치는 무한대 점수로 표시
        
        text_chars = list(normalized_text)  # Split into characters for proper Korean handling
        text_ngrams = set()
        
        # Create character-level n-grams
        for i in range(len(text_chars) - n + 1):
            ngram = ''.join(text_chars[i:i+n])
            text_ngrams.add(ngram)
        
        # 토큰 기반 n-gram 추가
        tokens = self._tokenize(normalized_text)
        for token in tokens:
            if len(token) >= n:
                text_ngrams.add(f"TOKEN:{token}")
        
        candidates = set()
        for ngram in text_ngrams:
            if ngram in self.ngram_index:
                candidates.update(self.ngram_index[ngram])
        
        # Prioritize candidates with multiple n-gram matches
        candidate_scores = {}
        for candidate in candidates:
            candidate_normalized = self._normalize_text(candidate)
            candidate_chars = list(candidate_normalized)
            candidate_ngrams = set()
            
            # 문자 n-gram
            for i in range(len(candidate_chars) - n + 1):
                ngram = ''.join(candidate_chars[i:i+n])
                candidate_ngrams.add(ngram)
            
            # 토큰 기반 n-gram
            candidate_tokens = self._tokenize(candidate_normalized)
            for token in candidate_tokens:
                if len(token) >= n:
                    candidate_ngrams.add(f"TOKEN:{token}")
            
            # n-gram 교집합 크기로 초기 점수 계산
            overlap = len(candidate_ngrams.intersection(text_ngrams))
            
            # 토큰 수준 유사도 보너스 점수 추가
            token_bonus = 0
            if self.token_similarity:
                query_tokens = set(tokens)
                cand_tokens = set(candidate_tokens)
                
                # 공통 토큰 비율 계산
                if query_tokens and cand_tokens:
                    common = query_tokens.intersection(cand_tokens)
                    token_bonus = len(common) * 2  # 토큰 일치에 높은 가중치 부여
            
            candidate_scores[candidate] = overlap + token_bonus
        
        # Return candidates sorted by n-gram overlap score
        return sorted(candidate_scores.items(), key=lambda x: x[1], reverse=True)
    
    def _calculate_similarity(self, text, entity):
        """다양한 유사도 측정 방법을 결합하여 더 정확한 유사도 계산"""
        normalized_text = self._normalize_text(text)
        normalized_entity = self._normalize_text(entity)
        
        # 정확히 일치하면 100점 반환
        if normalized_text == normalized_entity:
            return 100
        
        # 기본 문자열 유사도 (fuzz.ratio)
        ratio_score = fuzz.ratio(normalized_text, normalized_entity)
        
        # 부분 문자열 체크 (한 문자열이 다른 문자열의 부분 문자열인 경우)
        partial_score = 0
        if normalized_text in normalized_entity:
            text_len = len(normalized_text)
            entity_len = len(normalized_entity)
            partial_score = (text_len / entity_len) * 100 if entity_len > 0 else 0
        elif normalized_entity in normalized_text:
            text_len = len(normalized_text)
            entity_len = len(normalized_entity)
            partial_score = (entity_len / text_len) * 100 if text_len > 0 else 0
        
        # 토큰 유사도 (토큰 단위로 비교)
        token_score = 0
        if self.token_similarity:
            text_tokens = set(self._tokenize(normalized_text))
            entity_tokens = set(self._tokenize(normalized_entity))
            
            if text_tokens and entity_tokens:
                common_tokens = text_tokens.intersection(entity_tokens)
                all_tokens = text_tokens.union(entity_tokens)
                
                if all_tokens:
                    token_score = (len(common_tokens) / len(all_tokens)) * 100
        
        # 토큰 순서 무시 유사도 (fuzz.token_sort_ratio)
        token_sort_score = fuzz.token_sort_ratio(normalized_text, normalized_entity)
        
        # 토큰 집합 유사도 (fuzz.token_set_ratio)
        token_set_score = fuzz.token_set_ratio(normalized_text, normalized_entity)
        
        # 최종 유사도는 여러 점수의 가중 평균
        # 토큰 유사도에 높은 가중치를 부여하여 "T우주"와 "T 우주패스"의 매칭 향상
        final_score = (
            ratio_score * 0.3 +  # 기본 유사도
            max(partial_score, 0) * 0.1 +  # 부분 문자열 유사도
            token_score * 0.2 +  # 토큰 유사도
            token_sort_score * 0.2 +  # 토큰 순서 무시 유사도
            token_set_score * 0.2  # 토큰 집합 유사도
        )
        
        return final_score
    
    def find_entities(self, text, max_candidates_per_span=10):
        """Find entity matches in Korean text using fuzzy matching"""
        # Extract spans that might contain entities
        potential_spans = self._extract_korean_spans(text)
        matches = []
        
        for span_text, start, end in potential_spans:
            if len(span_text.strip()) < self.min_entity_length:  # Skip spans shorter than min_entity_length
                continue

            # Get candidate entities based on n-gram overlap
            candidates = self._get_candidates(span_text)

            # If no candidates found through n-gram filtering, skip
            if not candidates:
                continue
            
            # Limit the number of candidates to check
            top_candidates = [c[0] for c in candidates[:max_candidates_per_span]]
            
            # 각 후보 엔티티에 대해 개선된 유사도 계산
            scored_matches = []
            for entity in top_candidates:
                score = self._calculate_similarity(span_text, entity)
                
                if score >= self.min_similarity:
                    scored_matches.append((entity, score, 0))  # 호환성을 위해 3번째 매개변수 추가

            # 기존 process.extract 대신 개선된 유사도 계산 사용
            best_matches = scored_matches

            for entity, score, _ in best_matches:
                matches.append({
                    'text': span_text,
                    'matched_entity': entity,
                    'score': score,
                    'start': start,
                    'end': end,
                    'data': self.entity_data.get(entity, {})
                })

        # Sort by position in text
        matches.sort(key=lambda x: (x['start'], -x['score']))
        
        # Handle overlapping matches by keeping the best match
        final_matches = self._resolve_overlapping_matches(matches)

        return final_matches
    
    def _extract_korean_spans(self, text):
        """한국어와 영어가 혼합된 텍스트에서 엔티티일 수 있는 잠재적 텍스트 범위 추출"""
        spans = []
        min_len = self.min_entity_length
        
        # 1. 영문+한글 혼합 패턴 (붙여쓰기) 예: "T우주"
        for match in re.finditer(r'[a-zA-Z]+[가-힣]+(?:\s+[가-힣가-힣a-zA-Z0-9]+)*', text):
            if len(match.group(0)) >= min_len:
                spans.append((match.group(0), match.start(), match.end()))
        
        # 2. 영문+한글 혼합 패턴 (띄어쓰기) 예: "T 우주"
        for match in re.finditer(r'[a-zA-Z]+\s+[가-힣]+(?:\s+[가-힣가-힣a-zA-Z0-9]+)*', text):
            if len(match.group(0)) >= min_len:
                spans.append((match.group(0), match.start(), match.end()))
        
        # 3. 특수한 혼합 패턴 - 영문+한글+숫자 (예: "T우주365", "SK텔레콤")
        for match in re.finditer(r'[a-zA-Z]+[가-힣]+(?:[0-9]+)?', text):
            if len(match.group(0)) >= min_len:
                spans.append((match.group(0), match.start(), match.end()))
        
        # 4. 연속된 두 단어까지 확장 (예: "T우주 패스")
        # 영문+한글 후 공백 하나를 두고 다른 한글 단어가 나오는 패턴
        for match in re.finditer(r'[a-zA-Z]+[가-힣]+\s+[가-힣]+', text):
            if len(match.group(0)) >= min_len:
                spans.append((match.group(0), match.start(), match.end()))
                
        # 5. 연속된 세 단어까지 확장 (예: "T우주 멤버십 패스")
        for match in re.finditer(r'[a-zA-Z]+[가-힣]+\s+[가-힣]+\s+[가-힣]+', text):
            if len(match.group(0)) >= min_len:
                spans.append((match.group(0), match.start(), match.end()))
        
        # 6. 브랜드명 + 제품명 패턴 (예: "SK텔레콤 T우주")
        for match in re.finditer(r'[a-zA-Z가-힣]+(?:\s+[a-zA-Z가-힣]+){1,3}', text):
            if len(match.group(0)) >= min_len:
                spans.append((match.group(0), match.start(), match.end()))
        
        # 7. 숫자와 영문 결합 패턴 (숫자 space 영문 패턴, e.g. "0 day")
        for match in re.finditer(r'\d+\s+[a-zA-Z]+', text):
            if len(match.group(0)) >= min_len:
                spans.append((match.group(0), match.start(), match.end()))
        
        # 8. 더 일반적인 영한 혼합 패턴
        for match in re.finditer(r'[a-zA-Z가-힣0-9]+(?:\s+[a-zA-Z가-힣0-9]+)*', text):
            if len(match.group(0)) >= min_len:
                spans.append((match.group(0), match.start(), match.end()))
        
        # 9. 일반적인 구분자로 분리된 텍스트 조각도 추출
        for span in re.split(r'[,\.!?;:"\'…\(\)\[\]\{\}\s_/]+', text):
            if span and len(span) >= min_len:
                span_pos = text.find(span)
                if span_pos != -1:
                    spans.append((span, span_pos, span_pos + len(span)))
                
        return spans
    
    def _remove_duplicate_entities(self, matches):
        """Keep only one instance of each unique entity"""
        if not matches:
            return []
        
        # Dictionary to track highest-scoring match for each entity
        best_matches = {}
        
        for match in matches:
            entity_key = match['matched_entity']
            
            # If we haven't seen this entity before, or if this match has a higher score
            # than the previously saved match for this entity, save this one
            if (entity_key not in best_matches or 
                match['score'] > best_matches[entity_key]['score']):
                best_matches[entity_key] = match
        
        # Return the best matches sorted by start position
        return sorted(best_matches.values(), key=lambda x: x['start'])
    
    def _resolve_overlapping_matches(self, matches, high_score_threshold=50, overlap_tolerance=0.5):
        if not matches:
            return []
        
        # 점수 내림차순, 길이 오름차순으로 정렬 (높은 점수, 짧은 매치 우선)
        sorted_matches = sorted(matches, key=lambda x: (-x['score'], x['end'] - x['start']))
        
        final_matches = []
        
        for current_match in sorted_matches:
            current_score = current_match['score']
            current_start, current_end = current_match['start'], current_match['end']
            current_range = set(range(current_start, current_end))
            current_len = len(current_range)
            
            # 수정: overlap_ratio를 0으로 초기화
            current_match['overlap_ratio'] = 0.0
            
            # 높은 점수의 매치는 항상 포함
            if current_score >= high_score_threshold:
                # 기존 매치들과 비교하여 너무 많은 중복이 있는지 확인
                is_too_similar = False
                
                for existing_match in final_matches:
                    if existing_match['score'] < high_score_threshold:
                        continue  # 낮은 점수의 기존 매치와는 비교하지 않음
                        
                    existing_start, existing_end = existing_match['start'], existing_match['end']
                    existing_range = set(range(existing_start, existing_end))
                    
                    # 교집합 비율 계산
                    intersection = current_range.intersection(existing_range)
                    
                    # 현재 매치에 대한 중복 비율 
                    current_overlap_ratio = len(intersection) / current_len if current_len > 0 else 0
                    
                    # 수정: 중복 비율 저장 - 가장 높은 중복 비율 저장
                    current_match['overlap_ratio'] = max(current_match['overlap_ratio'], current_overlap_ratio)
                    
                    # 중복 비율이 허용 범위를 초과하고, 동일한 엔티티면 추가하지 않음
                    if (current_overlap_ratio > overlap_tolerance
                        and current_match['matched_entity'] == existing_match['matched_entity']
                        ):
                        is_too_similar = True
                        break
                
                if not is_too_similar:
                    final_matches.append(current_match)
            else:
                # 낮은 점수의 매치는 기존 로직 적용 (중복 확인)
                should_add = True
                
                for existing_match in final_matches:
                    existing_start, existing_end = existing_match['start'], existing_match['end']
                    existing_range = set(range(existing_start, existing_end))
                    
                    # 교집합 비율 계산
                    intersection = current_range.intersection(existing_range)
                    current_overlap_ratio = len(intersection) / current_len if current_len > 0 else 0
                    
                    # 수정: 중복 비율 저장 - 가장 높은 중복 비율 저장
                    current_match['overlap_ratio'] = max(current_match['overlap_ratio'], current_overlap_ratio)
                    
                    # 중복 비율이 허용 범위를 초과하면 추가하지 않음
                    if current_overlap_ratio > (1 - overlap_tolerance):
                        should_add = False
                        break
                
                if should_add:
                    final_matches.append(current_match)
        
        # 시작 위치별로 정렬
        final_matches.sort(key=lambda x: x['start'])
        
        return final_matches

def find_entities_in_text(text, entity_list, min_similarity=70, ngram_size=3, min_entity_length=2, 
                         token_similarity=True, high_score_threshold=50, overlap_tolerance=0.5):
    """
    Find entity matches in text using fuzzy matching.
    
    Parameters:
    -----------
    text : str
        The text to search for entities
    entity_list : list
        List of entities to match against
    min_similarity : int, default=70
        Minimum similarity score (0-100) for fuzzy matching
    ngram_size : int, default=2
        Size of character n-grams to use for indexing (2 or 3 recommended for Korean)
    min_entity_length : int, default=2
        Minimum length of entities to consider (characters)
    token_similarity : bool, default=True
        Whether to use token-based similarity measures
    high_score_threshold : int, default=50
        Score threshold above which matches are always kept regardless of overlap
    overlap_tolerance : float, default=0.5
        Overlap tolerance ratio (0-1), higher values allow more overlapping matches
        
    Returns:
    --------
    list
        List of matched entities with position and metadata
    """
    matcher = KoreanEntityMatcher(
        min_similarity=min_similarity,
        ngram_size=ngram_size,
        min_entity_length=min_entity_length,
        token_similarity=token_similarity
    )
    matcher.build_from_list(entity_list)
    
    matches = matcher.find_entities(text)
    
    # 기존 _resolve_overlapping_matches 메서드 대신 직접 호출
    final_matches = matcher._resolve_overlapping_matches(
        matches, 
        high_score_threshold=high_score_threshold,
        overlap_tolerance=overlap_tolerance
    )
    
    return final_matches
# Function to highlight entities in text
def highlight_entities(text, matches):
    marked_text = text
    offset = 0
    for match in sorted(matches, key=lambda x: x['start'], reverse=True):
        start = match['start'] + offset
        end = match['end'] + offset
        entity = match['matched_entity']
        score = match['score']
        marked_text = marked_text[:start] + f"[{marked_text[start:end]}→{entity} ({score:.1f}%)]" + marked_text[end:]
        offset += len(f"[→{entity} ({score:.1f}%)]") + 2
    
    return marked_text


In [5]:
import numpy as np
import matplotlib.pyplot as plt
from difflib import SequenceMatcher

def advanced_sequential_similarity(str1, str2, metrics=None, visualize=False):
    """
    Calculate multiple character-level similarity metrics between two strings.
    
    Parameters:
    -----------
    str1 : str
        First string
    str2 : str
        Second string
    metrics : list
        List of metrics to compute. Options: 
        ['ngram', 'lcs', 'subsequence', 'difflib']
        If None, all metrics will be computed
    visualize : bool
        If True, visualize the differences between strings
        
    Returns:
    --------
    dict
        Dictionary containing similarity scores for each metric
    """
    if metrics is None:
        metrics = ['ngram', 'lcs', 'subsequence', 'difflib']
    
    results = {}
    
    # Handle empty strings
    if not str1 or not str2:
        return {metric: 0.0 for metric in metrics}
    
    # Prepare strings
    s1, s2 = str1.lower(), str2.lower()
    
    # 1. N-gram similarity (with multiple window sizes)
    if 'ngram' in metrics:
        ngram_scores = {}
        for window in range(min([len(s1),len(s2),2]), min([5,max([len(s1),len(s2)])+1])):
            # Skip if strings are shorter than window
            if len(s1) < window or len(s2) < window:
                ngram_scores[f'window_{window}'] = 0.0
                continue
                
            # Generate character n-grams
            ngrams1 = [s1[i:i+window] for i in range(len(s1) - window + 1)]
            ngrams2 = [s2[i:i+window] for i in range(len(s2) - window + 1)]
            
            # Count matches
            matches = sum(1 for ng in ngrams1 if ng in ngrams2)
            max_possible = max(len(ngrams1), len(ngrams2))
            
            # Normalize
            score = matches / max_possible if max_possible > 0 else 0.0
            ngram_scores[f'window_{window}'] = score
            
        # Average of all n-gram scores
        results['ngram'] = max(ngram_scores.values())#sum(ngram_scores.values()) / len(ngram_scores)
        results['ngram_details'] = ngram_scores
    
    # 2. Longest Common Substring (LCS)
    if 'lcs' in metrics:
        def longest_common_substring(s1, s2):
            # Dynamic programming approach
            m, n = len(s1), len(s2)
            dp = [[0] * (n + 1) for _ in range(m + 1)]
            max_length = 0
            
            for i in range(1, m + 1):
                for j in range(1, n + 1):
                    if s1[i-1] == s2[j-1]:
                        dp[i][j] = dp[i-1][j-1] + 1
                        max_length = max(max_length, dp[i][j])
            
            return max_length
        
        lcs_length = longest_common_substring(s1, s2)
        max_length = max(len(s1), len(s2))
        results['lcs'] = lcs_length / max_length if max_length > 0 else 0.0
    
    # 3. Longest Common Subsequence
    if 'subsequence' in metrics:
        def longest_common_subsequence(s1, s2):
            # Dynamic programming approach for subsequence
            m, n = len(s1), len(s2)
            dp = [[0] * (n + 1) for _ in range(m + 1)]
            
            for i in range(1, m + 1):
                for j in range(1, n + 1):
                    if s1[i-1] == s2[j-1]:
                        dp[i][j] = dp[i-1][j-1] + 1
                    else:
                        dp[i][j] = max(dp[i-1][j], dp[i][j-1])
            
            return dp[m][n]
        
        subseq_length = longest_common_subsequence(s1, s2)
        max_length = max(len(s1), len(s2))
        results['subsequence'] = subseq_length / max_length if max_length > 0 else 0.0
    
    # 4. SequenceMatcher from difflib
    if 'difflib' in metrics:
        sm = SequenceMatcher(None, s1, s2)
        results['difflib'] = sm.ratio()
    
    # Visualization of differences
    if visualize:
        try:
            # Only works in notebooks or environments that support plotting
            sm = SequenceMatcher(None, s1, s2)
            matches = sm.get_matching_blocks()
            
            # Prepare for visualization
            fig, ax = plt.subplots(figsize=(10, 3))
            
            # Draw strings as horizontal bars
            ax.barh(0, len(s1), height=0.4, left=0, color='lightgray', alpha=0.3)
            ax.barh(1, len(s2), height=0.4, left=0, color='lightgray', alpha=0.3)
            
            # Draw matching parts
            for match in matches:
                i, j, size = match
                if size > 0:  # Ignore zero-length matches
                    ax.barh(0, size, height=0.4, left=i, color='green', alpha=0.5)
                    ax.barh(1, size, height=0.4, left=j, color='green', alpha=0.5)
                    
                    # Draw connection lines between matches
                    ax.plot([i + size/2, j + size/2], [0.2, 0.8], 'k-', alpha=0.3)
            
            # Add string texts
            for i, c in enumerate(s1):
                ax.text(i + 0.5, 0, c, ha='center', va='center')
            for i, c in enumerate(s2):
                ax.text(i + 0.5, 1, c, ha='center', va='center')
                
            ax.set_yticks([0, 1])
            ax.set_yticklabels(['String 1', 'String 2'])
            ax.set_xlabel('Character Position')
            ax.set_title('Character-Level String Comparison')
            ax.grid(False)
            plt.tight_layout()
            # plt.show()  # Uncomment to display
        except Exception as e:
            print(f"Visualization error: {e}")
    
    # Calculate overall similarity score (average of all metrics)
    metrics_to_average = [m for m in results.keys() if not m.endswith('_details')]
    results['overall'] = sum(results[m] for m in metrics_to_average) / len(metrics_to_average)
    
    return results

# advanced_sequential_similarity('시크릿', '시크릿', metrics='ngram')
# advanced_sequential_similarity('에이닷_자사', '에이닷')

In [6]:
import pandas as pd

item_pdf_raw = pd.read_csv("./data/item_info_all_250527.csv")

item_pdf_all = item_pdf_raw.drop_duplicates(['item_nm','item_id'])[['item_nm','item_id','item_desc','domain','start_dt','end_dt','rank']].copy()

# item_pdf_all.query("rank<1000")[['item_nm']].drop_duplicates().to_csv("./data/item_nm_1000.csv", index=False)
alias_pdf = pd.read_csv("./data/alias_rules.csv")
alia_rule_set = list(zip(alias_pdf['alias_1'], alias_pdf['alias_2']))

def apply_alias_rule(item_nm):
    item_nm_list = [item_nm]

    for r in alia_rule_set:
        if r[0] in item_nm:
            item_nm_list.append(item_nm.replace(r[0], r[1]))
        if r[1] in item_nm:
            item_nm_list.append(item_nm.replace(r[1], r[0]))
    return item_nm_list

item_pdf_all['item_nm_alias'] = item_pdf_all['item_nm'].apply(apply_alias_rule)

item_pdf_all = item_pdf_all.explode('item_nm_alias')

item_pdf_all.query("rank<1000 and item_nm.str.contains('넷플릭스', case=False) or item_nm.str.contains('웨이브', case=False)")[['item_nm','item_nm_alias','item_id']]

user_defined_entity = ['AIA Vitality' , '부스트 파크 건대입구' , 'Boost Park 건대입구']
item_pdf_ext = pd.DataFrame([{'item_nm':e,'item_id':e,'item_desc':e, 'domain':'user_defined', 'start_dt':20250101, 'end_dt':99991231, 'rank':1, 'item_nm_alias':e} for e in user_defined_entity])
item_pdf_all = pd.concat([item_pdf_all,item_pdf_ext])

stop_item_names = pd.read_csv("./data/stop_words.csv")['stop_words'].to_list()

entity_vocab = []
for row in item_pdf_all.to_dict('records'):
    if row['item_nm_alias'] in stop_item_names:
        continue
    entity_vocab.append((row['item_nm_alias'], {'item_nm':row['item_nm'], 'item_id':row['item_id'], 'description':row['item_desc'], 'domain':row['domain'],'item_nm_alias':row['item_nm_alias']}))


entity_list_for_fuzzy = []
for row in item_pdf_all.to_dict('records'):
    entity_list_for_fuzzy.append((row['item_nm_alias'], {'item_nm':row['item_nm'], 'item_id':row['item_id'], 'description':row['item_desc'], 'domain':row['domain'], 'start_dt':row['start_dt'], 'end_dt':row['end_dt'], 'rank':1, 'item_nm_alias':row['item_nm_alias']}))


In [7]:
mms_pdf = pd.read_csv("./data/mms_data_250408.csv")
mms_pdf['msg'] = mms_pdf['msg_nm']+"\n"+mms_pdf['mms_phrs']
mms_pdf = mms_pdf.groupby(["msg_nm","mms_phrs","msg"])['offer_dt'].min().reset_index(name="offer_dt")
mms_pdf = mms_pdf.reset_index()
mms_pdf = mms_pdf.astype('str')

In [8]:
from sentence_transformers import SentenceTransformer
import torch
emb_model = SentenceTransformer('jhgan/ko-sbert-nli')

def preprocess_text(text):
    # 특수문자를 공백으로 변환
    text = re.sub(r'[^\w\s]', ' ', text)
    # 여러 공백을 하나로 통일
    text = re.sub(r'\s+', ' ', text)
    # 앞뒤 공백 제거
    return text.strip()

item_embeddings = emb_model.encode([x.lower() for x in item_pdf_all['item_nm'].tolist()], convert_to_tensor=True)

In [9]:

import re

num_cand_pgms = 5

pgm_pdf = pd.read_csv("./data/pgm_tag_ext_250516.csv")

clue_embeddings = emb_model.encode(pgm_pdf[["pgm_nm","clue_tag"]].apply(lambda x: preprocess_text(x['pgm_nm'].lower())+" "+x['clue_tag'].lower(), axis=1).tolist(), convert_to_tensor=True)


In [10]:
ner = KoreanNERSpacy(entity_vocab)


In [11]:

for test_text in mms_pdf.sample(10)['msg'].tolist():
# for test_text in msg_text_list:
    print(f"Test text: {test_text.strip()}")

    matches = []
    matches.extend(find_entities_in_text(
            test_text.strip(), 
            entity_list_for_fuzzy, 
            min_similarity=60,
            high_score_threshold=50,
            overlap_tolerance=0.5
        ))

    mdf = pd.DataFrame(matches)
    

    print("fuzzy_old: ",', '.join(set([f"{p['matched_entity']} ({p['data']['domain']})" for p in matches if p['matched_entity'] not in stop_item_names and p['text'] not in stop_item_names])))
    print()

    # Test each method
    methods = ["fuzzy"]

    for method in methods:
        print()
        results = ner.extract_entities(test_text, method)
        
        # print(f"Found {len(results)} unique entities:")
        
        # Group by canonical name to show deduplication
        canonical_groups = {}
        for result in results:
            canonical = result['canonical_name']

            if canonical in stop_item_names:
                continue
            
            if canonical not in canonical_groups:
                canonical_groups[canonical] = []
            canonical_groups[canonical].append(result)

        print(method+": ",', '.join([f"{canonical} ({group[0]['domain']})" for canonical, group in canonical_groups.items()]))
    print()

    break
    


Test text: 인천중구대리점  부천역직영점 오픈 안내
(광고)[SKT] 인천중구대리점  부천역직영점 오픈 안내  고객님, 안녕하세요. SK텔레콤이 부천역 4번출구 알라딘 문구 1층에 새로운 T프리미엄스토어를 열었습니다. ★매장위치 확인:  http://t-mms.kr/t.do?m=#61&u=http://naver.me/FaV6yr6v ★  ★우리 매장 친구맺기 혜택 안내★ http://t-mms.kr/t.do?m=#61&u=https://tworldfriends.co.kr/d131240160    ■ 오픈 기념 혜택 - 휴대폰구매고객: 6종 사은품   ① 저주파 무선 안마기   ② 카카오타월   ③ 그릇셋트   ④ 카카오무선선풍기   ⑤ 올인원케이블(모든타입 사용 가능)   ⑥ 고급 고속 충전기 - 인터넷/TV 설치시 상품권 최대 40만원 - 단골등록고객: 스마트폰 터치펜  또는 그립톡증정 및 휴대폰 액정필름 무료교체 ※ 위 사은품은 조기 종료 또는 대체될 수 있습니다.  ★ 혜택 내용 자세히 보기:  http://t-mms.kr/t.do?m=#61&u=http://www.sktbestshop.com/ * 이 문자를 받으신 고객님만 혜택을 받으실 수 있습니다. ------------------------------------------------------------------  ■ 매장위치 및 영업시간 안내 - 주소 : 경기 부천시 부천로4(경동빌딩) - 연락처 : 032-887-0001 (문의사항 있을시 매장연락처로 문의주십시오) - 영업시간 : 평일 오전 10시~오후 8시 (주말 오전11시~오후7시)  SKT와 함께해주셔서 감사합니다.  무료 수신거부 1504
fuzzy_old:  MDL-0001 (mobile_device), T 연락처 (product), 티봇_887 (product), 티봇_0001 (product)


fuzzy:  카카오 (affiliated_service)



In [12]:
msg_text_list = ["""
광고 제목:[SK텔레콤] 2월 0 day 혜택 안내
광고 내용:(광고)[SKT] 2월 0 day 혜택 안내__[2월 10일(토) 혜택]_만 13~34세 고객이라면_베어유 모든 강의 14일 무료 수강 쿠폰 드립니다!_(선착순 3만 명 증정)_▶ 자세히 보기: http://t-mms.kr/t.do?m=#61&s=24589&a=&u=https://bit.ly/3SfBjjc__■ 에이닷 X T 멤버십 시크릿코드 이벤트_에이닷 T 멤버십 쿠폰함에 ‘에이닷이빵쏜닷’을 입력해보세요!_뚜레쥬르 데일리우유식빵 무료 쿠폰을 드립니다._▶ 시크릿코드 입력하러 가기: https://bit.ly/3HCUhLM__■ 문의: SKT 고객센터(1558, 무료)_무료 수신거부 1504
""",
"""
광고 제목:통화 부가서비스를 패키지로 저렴하게!
광고 내용:(광고)[SKT] 콜링플러스 이용 안내  #04 고객님, 안녕하세요. <콜링플러스>에 가입하고 콜키퍼, 컬러링, 통화가능통보플러스까지 총 3가지의 부가서비스를 패키지로 저렴하게 이용해보세요.  ■ 콜링플러스 - 이용요금: 월 1,650원, 부가세 포함 - 콜키퍼(550원), 컬러링(990원), 통화가능통보플러스(770원)를 저렴하게 이용할 수 있는 상품  ■ 콜링플러스 가입 방법 - T월드 앱: 오른쪽 위에 있는 돋보기를 눌러 콜링플러스 검색 > 가입  ▶ 콜링플러스 가입하기: http://t-mms.kr/t.do?m=#61&u=https://skt.sh/17tNH  ■ 유의 사항 - 콜링플러스에 가입하면 기존에 이용 중인 콜키퍼, 컬러링, 통화가능통보플러스 서비스는 자동으로 해지됩니다. - 기존에 구매한 컬러링 음원은 콜링플러스 가입 후에도 계속 이용할 수 있습니다.(시간대, 발신자별 설정 정보는 다시 설정해야 합니다.)  * 최근 다운로드한 음원은 보관함에서 무료로 재설정 가능(다운로드한 날로부터 1년 이내)   ■ 문의: SKT 고객센터(114)  SKT와 함께해주셔서 감사합니다.  무료 수신거부 1504\n    ', 
""",
"""
(광고)[SKT] 1월 0 day 혜택 안내_ _[1월 20일(토) 혜택]_만 13~34세 고객이라면 _CU에서 핫바 1,000원에 구매 하세요!_(선착순 1만 명 증정)_▶ 자세히 보기 : http://t-mms.kr/t.do?m=#61&s=24264&a=&u=https://bit.ly/3H2OHSs__■ 에이닷 X T 멤버십 구독캘린더 이벤트_0 day 일정을 에이닷 캘린더에 등록하고 혜택 날짜에 알림을 받아보세요! _알림 설정하면 추첨을 통해 [스타벅스 카페 라떼tall 모바일쿠폰]을 드립니다. _▶ 이벤트 참여하기 : https://bit.ly/3RVSojv_ _■ 문의: SKT 고객센터(1558, 무료)_무료 수신거부 1504
""",
"""
'[T 우주] 넷플릭스와 웨이브를 월 9,900원에! \n(광고)[SKT] 넷플릭스+웨이브 월 9,900원, 이게 되네! __#04 고객님,_넷플릭스와 웨이브 둘 다 보고 싶었지만, 가격 때문에 망설이셨다면 지금이 바로 기회! __오직 T 우주에서만, _2개월 동안 월 9,900원에 넷플릭스와 웨이브를 모두 즐기실 수 있습니다.__8월 31일까지만 드리는 혜택이니, 지금 바로 가입해 보세요! __■ 우주패스 Netflix 런칭 프로모션 _- 기간 : 2024년 8월 31일(토)까지_- 혜택 : 우주패스 Netflix(광고형 스탠다드)를 2개월 동안 월 9,900원에 이용 가능한 쿠폰 제공_▶ 프로모션 자세히 보기: http://t-mms.kr/jAs/#74__■ 우주패스 Netflix(월 12,000원)  _- 기본 혜택 : Netflix 광고형 스탠다드 멤버십_- 추가 혜택 : Wavve 콘텐츠 팩 _* 추가 요금을 내시면 Netflix 스탠다드와 프리미엄 멤버십 상품으로 가입 가능합니다.  __■ 유의 사항_-  프로모션 쿠폰은 1인당 1회 다운로드 가능합니다. _-  쿠폰 할인 기간이 끝나면 정상 이용금액으로 자동 결제 됩니다. __■ 문의: T 우주 고객센터 (1505, 무료)__나만의 구독 유니버스, T 우주 __무료 수신거부 1504'
""",
"""
광고 제목:[SK텔레콤] T건강습관 X AIA Vitality, 우리 가족의 든든한 보험!
광고 내용:(광고)[SKT] 가족의 든든한 보험 (무배당)AIA Vitality 베스트핏 보장보험 안내  고객님, 안녕하세요. 4인 가족 표준생계비, 준비하고 계시나요? (무배당)AIA Vitality 베스트핏 보장보험(디지털 전용)으로 최대 20% 보험료 할인과 가족의 든든한 보험 보장까지 누려 보세요.   ▶ 자세히 보기: http://t-mms.kr/t.do?m=#61&u=https://bit.ly/36oWjgX  ■ AIA Vitality  혜택 - 매달 리워드 최대 12,000원 - 등급 업그레이드 시 특별 리워드 - T건강습관 제휴 할인 최대 40% ※ 제휴사별 할인 조건과 주간 미션 달성 혜택 등 자세한 내용은 AIA Vitality 사이트에서 확인하세요. ※ 이 광고는 AIA생명의 광고이며 SK텔레콤은 모집 행위를 하지 않습니다.  - 보험료 납입 기간 중 피보험자가 장해분류표 중 동일한 재해 또는 재해 이외의 동일한 원인으로 여러 신체 부위의 장해지급률을 더하여 50% 이상인 장해 상태가 된 경우 차회 이후의 보험료 납입 면제 - 사망보험금은 계약일(부활일/효력회복일)로부터 2년 안에 자살한 경우 보장하지 않음 - 일부 특약 갱신 시 보험료 인상 가능 - 기존 계약 해지 후 신계약 체결 시 보험인수 거절, 보험료 인상, 보장 내용 변경 가능 - 해약 환급금(또는 만기 시 보험금이나 사고보험금)에 기타 지급금을 합해 5천만 원까지(본 보험 회사 모든 상품 합산) 예금자 보호 - 계약 체결 전 상품 설명서 및 약관 참조 - 월 보험료 5,500원(부가세 포함)  * 생명보험협회 심의필 제2020-03026호(2020-09-22) COM-2020-09-32426  ■문의: 청약 관련(1600-0880)  무료 수신거부 1504    
"""
]

message_idx = 1

test_text = msg_text_list[message_idx]


In [24]:
schema_prd = {
    "title": {
        "type": "string", 
        'description': '광고 제목. 광고의 핵심 주제와 가치 제안을 명확하게 설명할 수 있도록 생성'
    },
    'purpose': {
        'type': 'array', 
        'description': '광고의 주요 목적을 다음 중에서 선택(복수 가능): [상품 가입 유도, 대리점/매장 방문 유도, 웹/앱 접속 유도, 이벤트 응모 유도, 혜택 안내, 쿠폰 제공 안내, 경품 제공 안내, 수신 거부 안내, 기타 정보 제공]'
    },
    'product': {
        'type': 'array',
        'items': {
            'type': 'object',
            'properties': {
            'name': {'type': 'string', 'description': '광고하는 제품이나 서비스 이름'},
            # 'category': {'type': 'string', 'description': '광고 상품의 카테고리. [요금제, 부가서비스, 구독서비스, 제휴서비스, 모바일 단말기, 기타] 중에서 선택'},
            'position': {'type': 'string', 'description': '광고 상품의 분류. [main, sub] 중에서 선택'},
            'action': {'type': 'string', 'description': '고객에게 기대하는 행동: [구매, 가입, 사용, 방문, 참여, 코드입력, 쿠폰다운로드, 기타] 중에서 선택'}
            }
        }
    },
    'channel': {
        'type': 'array', 
        'items': {
            'type': 'object', 
            'properties': {
                'type': {'type': 'string', 'description': '채널 종류: [URL, 전화번호, 앱, 대리점] 중에서 선택'},
                'value': {'type': 'string', 'description': '실제 URL, 전화번호, 앱 이름, 대리점 이름 등 구체적 정보'},
                'action': {'type': 'string', 'description': '채널 목적: [가입, 추가 정보, 문의, 수신, 수신 거부] 중에서 선택'},
                # 'store_code': {'type': 'string', 'description': "매장 코드 - tworldfriends.co.kr URL에서 D+숫자 9자리(D[0-9]{9}) 패턴의 코드 추출하여 대리점 채널에 설정"}
            }
        }
    },
    'pgm':{
        'type': 'array', 
        'description': '아래 광고 분류 기준 정보에서 선택. 메세지 내용과 광고 분류 기준을 참고하여, 광고 메세지에 가장 부합하는 2개의 pgm_nm을 적합도 순서대로 제공'
    },
}


schema_prompt = f"""
    아래와 같은 스키마로 결과를 제공해 주세요.

    {schema_prd}
"""


for test_text in mms_pdf.sample(10)['msg'].tolist():
    print(f"Test text: {test_text.strip()}")
    msg = test_text.strip()

    mms_embedding = emb_model.encode([msg.lower()], convert_to_tensor=True)

    similarities = torch.nn.functional.cosine_similarity(
        mms_embedding,  
        clue_embeddings,  
        dim=1 
    ).cpu().numpy()

    pgm_pdf_tmp = pgm_pdf.copy()
    pgm_pdf_tmp['sim'] = similarities

    pgm_pdf_tmp = pgm_pdf_tmp.sort_values('sim', ascending=False)

    pgm_cand_info = "\n\t".join(pgm_pdf_tmp.iloc[:num_cand_pgms][['pgm_nm','clue_tag']].apply(lambda x: re.sub(r'\[.*?\]', '', x['pgm_nm'])+" : "+x['clue_tag'], axis=1).to_list())
    rag_context = f"\n### 광고 분류 기준 정보 ###\n\t{pgm_cand_info}" if num_cand_pgms>0 else ""


    prompt = f"""
    아래 광고 메시지에서 광고 목적과 상품 이름을 추출해 주세요.

    ### 광고 메시지 ###
    {msg}

    ### 추출 작업 순서 ###
    1. 광고 목적을 먼저 파악한다.
    2. 파악된 목적에 기반하여 Main 상품을 추출한다.
    3. 추출한 Main 상품에 관련되는 Sub 상품을 추출한다.
    4. 추출된 상품 정보를 고려하여 채널 정보를 제공한다.

    ### 추출 작업 가이드 ###
    * 상품 추출시 정확도(precision) 보다는 재현율(recall)에 중심을 두어라.
    * 광고 목적에 대리점 방문이 포함되어 있으면 대리점 채널 정보를 제공해라.


    {schema_prompt}

    {rag_context}

    """

    print()

    # result_json_text = llm_cld40.invoke(prompt).content
    result_json_text = llm_gem3.invoke(prompt).content

    json_objects = extract_json_objects(result_json_text)[0]

    item_info_list = []
    for item in json_objects['product']:

        if item['name'] in stop_item_names:
            continue

        _embedding = emb_model.encode([item['name'].lower()], convert_to_tensor=True)

        similarities = torch.nn.functional.cosine_similarity(
            _embedding,  
            item_embeddings,  
            dim=1 
        ).cpu().numpy()

        item_pdf_tmp = item_pdf_all.copy()
        item_pdf_tmp['sim'] = similarities.round(5)
        item_pdf_tmp = item_pdf_tmp.drop_duplicates(subset=['item_nm','item_id']).query("sim>=0.8")

        item_pdf_tmp = item_pdf_tmp.groupby(['item_nm','sim'])['item_id'].apply(list).reset_index(name="item_id").sort_values('sim', ascending=False)

        sim_item_list = item_pdf_tmp[['item_nm','item_id']].rename(columns={'item_nm':'item_name_in_voca'}).head(5).to_dict(orient='records')

        item_info_list.append({"item_name_in_msg":item['name'],"item_in_voca":sim_item_list})

        # break

    final_result = json_objects.copy()
    final_result['product'] = item_info_list

    if num_cand_pgms>0:
        pgm_json = pgm_pdf[pgm_pdf['pgm_nm'].apply(lambda x: re.sub(r'\[.*?\]', '', x) in ' '.join(json_objects['pgm']))][['pgm_nm','pgm_id']].to_dict('records')
        final_result['pgm'] = pgm_json

    print("==="*15+"claude sonnet (emb)"+"==="*15+"\n")
    print(json.dumps(final_result, indent=4, ensure_ascii=False))


    matches = []
    for item_name_message in json_objects['product']:
        matches.extend([x for x in [{**e[1], 'entity_in_msg':item_name_message['name'], "sim":advanced_sequential_similarity(e[0], item_name_message['name'], metrics='difflib')} for e in entity_list_for_fuzzy] if x['sim']['difflib']>=0.75])

    mdf = pd.DataFrame([p for p in matches if p['entity_in_msg'] in select_most_comprehensive([p['entity_in_msg'] for p in matches])]).drop_duplicates(['item_nm','item_id','entity_in_msg'])

    if len(matches)>0:
        mdf = mdf.query("item_nm_alias.str.lower() not in @stop_item_names or entity_in_msg.str.lower() not in @stop_item_names")
        mdf['sim'] = mdf['sim'].apply(lambda x: x['difflib'])

    if mdf.shape[0]>0:
        mdf['rank'] = mdf.groupby('entity_in_msg')['sim'].rank(method='dense',ascending=False)
        mdf = mdf.query("rank<=5")

        product_tag = [{"item_name_in_msg":entity_in_msg, "item_in_voca":mdf.query("entity_in_msg==@entity_in_msg").groupby(['item_nm','sim'])['item_id'].apply(list).reset_index(name="item_id").sort_values('sim', ascending=False).drop('sim', axis=1).to_dict(orient='records')} for entity_in_msg in mdf['entity_in_msg'].unique()]

        final_result = {
            "title":json_objects['title'],
            "purpose":json_objects['purpose'],
            "product":product_tag,
            "channel":json_objects['channel'],
            "pgm":json_objects['pgm']
        }

    else:
        final_result = json_objects.copy()
        final_result['product'] = [{'item_name_in_msg':d['name'], 'item_in_voca':[{'item_name_in_voca':d['name'], 'item_id': ['#']}]} for d in final_result['product']]

    if num_cand_pgms>0:
        pgm_json = pgm_pdf[pgm_pdf['pgm_nm'].apply(lambda x: re.sub(r'\[.*?\]', '', x) in ' '.join(json_objects['pgm']))][['pgm_nm','pgm_id']].to_dict('records')
        final_result['pgm'] = pgm_json


    print("==="*15+"claude sonnet (fuzzy)"+"==="*15+"\n")
    print(json.dumps(final_result, indent=4, ensure_ascii=False))

    print("\n\n")

    break


Test text: [SK텔레콤] 와이즈대리점 태평점 행사안내
(광고)[SKT]와이즈대리점 태평직영점 갤럭시 S23 출시 이벤트__* 단골 문자 받으신 고객님 매장 방문해주시면 따뜻한 핫팩 증정!!__SK텔레콤 공식인증 태평직영점에서 갤럭시 S23 출시 이벤트 안내드립니다.__■ 갤럭시 S23 구매 시 갤럭시 워치 또는 60만 원 상당 삼성노트북(갤럭시북GO) 증정_ - 갤럭시 S23 구매 시 최대 128만 원 할인(삼성보상, 제휴 카드 할인)_ - 24개월 후 반납 시 새 휴대폰으로 교체가능__■ 포켓몬 에디션 키즈폰 한정판 출시!! 새학기 사은품 증정!!_ - 태평동 당근마켓에서 많은 판매를 기록한 초등학생용폰__■SKT B tv air 이벤트_- 유선+휴대폰 가족 결합 3인 시 테블릿PC+넷플릭스등 OTT서비스(일부 요금 발생) 시청 가능__■ SK쉴더스(ADT캡스) 6개월 무료 체험 이벤트(선착순 10명)_ - 집앞 CCTV +도난, 화재보험_ - 무료 체험 후 불필요 시 해지 가능하며, 설치 및 해지 시 별도 비용 발생하지 않습니다._  _■ 인터넷+IPTV 가입 시 최대 사은품 증정__인터넷과 IPTV 가입 시 최대 사은품! 테블릿PC로 B tv 시청 무료 체험권!_항상 정직하고 친절한 SK텔레콤 직영점이 되겠습니다.__▶SKT 와이즈 태평점_- 주소: 경기도 성남시 수정구 수정로 119번길 4_- 연락처: 070-7865-8708__▶ 매장홈페이지: http://t-mms.kr/t.do?m=#61&s=18305&a=&u=https://tworldfriends.co.kr/D136650122__SK텔레콤과 함께해주셔서 감사합니다.__무료 수신거부 1504

=============================================claude sonnet (emb)=============================================

{
    "title": "갤럭시 S23 출시 기념 특별 이벤트 및 SKT 다양한 혜택 안내",
 